# Tracking MCMC Workflow
This notebook attempts to track the workflow of MCMC. This notebook has MANY purposes.

First, we inject a signal. For the injection, MCMC uses the following workflow.

MCMC calls ===> LALInferenceInjectInspiralSignal which calls ===>
[XLALSimInspiralChooseTDWaveform](http://software.ligo.org/docs/lalsuite/lalsimulation/group___l_a_l_sim_inspiral__c.html#gaf8e3028cce411328ddc1a3efa6f43cb2) ==>
does a giant switch..case.. over all the possible approximants. For a precessing waveform such as SpinTaylorT4 the inclination S1 and S2 from the injection XML gets passed through
[XLALSimInspiralInitialConditionsPrecessingApproxs](http://software.ligo.org/docs/lalsuite/lalsimulation/group___l_a_l_sim_inspiral_spin_taylor__c.html#gadc3d6ebbe9e62c8d7ea93b489db00bdb) which expects by default Input: x, y, z components S1 and S2 wrt reference L for axisChoice = LAL_SIM_INSPIRAL_FRAME_AXIS_ORBITAL_L/L and N (Nx ∝sin(inc), Ly=0) for axisChoice = LAL_SIM_INSPIRAL_FRAME_AXIS_ORBITAL_L (default) and outputs S1 and S2 Output: x, y, z components S1 and S2 wrt N and inc angle between L and N as required by precessing waveforms routines. This then gets passed into the waveform driver routine
[XLALSimInspiralSpinTaylorT4](http://software.ligo.org/docs/lalsuite/lalsimulation/group___l_a_l_sim_inspiral_spin_taylor__c.html#ga43f8a11e72b41797fef44c7b2134a1ba)

Second for the template

[LALInferenceTemplateXLALSimInspiralChooseWaveform](http://software.ligo.org/docs/lalsuite/lalinference/group___l_a_l_inference_template__h.html#ga8725ff30be0c84ea6be1e1cfb3a317ea) ==>
[LALSimInspiralTransformPrecessingNewInitialConditions](http://software.ligo.org/docs/lalsuite/lalsimulation/group___l_a_l_sim_inspiral_spin_taylor__c.html#gacde647cce8c72a41d1dd0ac9e9b385da) ==>
[XLALSimInspiralChooseTDWaveformFromCache](http://software.ligo.org/docs/lalsuite/lalsimulation/group___l_a_l_sim_inspiral_waveform_cache__h.html#ga21112bbad5c769738c19ddb56701e6ae)
==> [XLALSimInspiralChooseTDWaveform](http://software.ligo.org/docs/lalsuite/lalsimulation/group___l_a_l_sim_inspiral__c.html#gaf8e3028cce411328ddc1a3efa6f43cb2) ==> [XLALSimInspiralInitialConditionsPrecessingApproxs](http://software.ligo.org/docs/lalsuite/lalsimulation/group___l_a_l_sim_inspiral_spin_taylor__c.html#gadc3d6ebbe9e62c8d7ea93b489db00bdb)


The inclination,S1 S2 that are output by [LALSimInspiralTransformPrecessingNewInitialConditions](http://software.ligo.org/docs/lalsuite/lalsimulation/group___l_a_l_sim_inspiral_spin_taylor__c.html#gacde647cce8c72a41d1dd0ac9e9b385da) 
say they are in the right frame that can be passed into [XLALSimInspiralInitialConditionsPrecessingApproxs](http://software.ligo.org/docs/lalsuite/lalsimulation/group___l_a_l_sim_inspiral_spin_taylor__c.html#gadc3d6ebbe9e62c8d7ea93b489db00bdb)
i.e. look at the final rotation done in the code.
 * 3466         /* Rotation 6: Now L is along z and we have to bring N
 * 3467          * in the x-z plane.
 * 3468          */
 * 3469         REAL8 phiN = atan2(Ny, Nx);
 * 3470         ROTATEZ(-phiN, s1hatx, s1haty, s1hatz);
 * 3471         ROTATEZ(-phiN, s2hatx, s2haty, s2hatz);
 * 3472 
 * 3473         /* Set pointers to rotated spin vectors */
 * 3474         *S1x = s1hatx*chi1;
 * 3475         *S1y = s1haty*chi1;
 * 3476         *S1z = s1hatz*chi1;
 * 3477         *S2x = s2hatx*chi2;
 * 3478         *S2y = s2haty*chi2;
 * 3479         *S2z = s2hatz*chi2;
IT IS UNCLEAR IF THIS IS TRUE!!!!!!!!

In fact there is a check to make sure this is true

 3481         //Uncomment the following lines for a check of the rotation
 3482         /*printf("LNhat should be along z, N in the x-z plane\n");
 3483         ROTATEZ(-phiL,    LNhx, LNhy, LNhz);
 3484         ROTATEY(-thetaLJ, LNhx, LNhy, LNhz);
 3485         ROTATEZ(-phiN,    LNhx, LNhy, LNhz);
 3486         ROTATEZ(-phiN,    Nx, Ny, LNz);
 3487         ROTATEZ(-phi0, Jhatx, Jhaty, Jhatz);
 3488         ROTATEY(-theta0, Jhatx, Jhaty, Jhatz);
 3489         ROTATEZ(phiJL - LAL_PI, Jhatx, Jhaty, Jhatz);
 3490         ROTATEZ(-phiL, Jhatx, Jhaty, Jhatz);
 3491         ROTATEY(-thetaLJ, Jhatx, Jhaty, Jhatz);
 3492         printf("LNhat: %12.4e  %12.4e  %12.4e\n",LNhx,LNhy,LNhz);
 3493         printf("       %12.4e  %12.4e  %12.4e\n",0.,0.,1.);
 3494         printf("N:     %12.4e  %12.4e  %12.4e\n",Nx,Ny,Nz);
 3495         printf("       %12.4e  %12.4e  %12.4e\n",-sin(*incl),0.,cos(*incl));
 3496         printf("J.Lhat  i: %12.4e f: %12.4e\n",Jz,Jnorm*Jhatz);
 3497         printf("S1.L    i: %12.4e f: %12.4e\n",chi1*cos(theta1),*S1z);
 3498         printf("S2.L    i: %12.4e f: %12.4e\n",chi2*cos(theta2),*S2z);
 3499         printf("S1.S2   i: %12.4e f: %12.4e\n",chi1*chi2*(sin(theta1)*sin(theta2)*cos(phi12)+cos(theta1)*cos(theta2)),(*S1x)*(*S2x)+(*S1y)*(*S2y)+(*S1z)*(*S2z));*/